# Import Packages

In [166]:
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
import math
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, mean_squared_error
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
import gensim
import string
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
import warnings
from keras import backend as K

#K.tensorflow_backend._get_available_gpus()

# Import all brands' lotion review datasets

For our analysis purpose, only 'review rating','review text' should be kept. The usage of 'review rating' is to help classifying the same rating between different brands.  

In [167]:
df_Aquaphor = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Aquaphor Baby Healing Ointment Reviews.csv', skipinitialspace=True, usecols=['review text','review rating'])
df_Aveeno = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Aveeno Eczema Therapy Reviews.csv', skipinitialspace=True, usecols=['review text','review rating'])
df_CeraVe = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\CeraVe Baby Moisturizing Cream reviews.csv', skipinitialspace=True, usecols=['review text','review rating'])
df_Cetaphil = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Cetaphil Baby Eczema Soothing Lotion Reviews.csv', skipinitialspace=True, usecols=['review text','review rating'])
df_Era = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Era Organics Healing Ointment Reviews.csv', skipinitialspace=True, usecols=['review text','review rating'])
df_Eucerin = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Eucerin Eczema Relief Creme Review.csv', skipinitialspace=True, usecols=['review text','review rating'])
df_Mustela = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Mustela Stelatopia Reviews.csv', skipinitialspace=True, usecols=['review text','review rating'])
df_Vanicream = pd.read_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Vanicream Reviews.csv', skipinitialspace=True, usecols=['review text','review rating'])

In [168]:
df_Vanicream

,review rating,review text
0,2,Originally our pediatrician recommended this l...
1,5,This wash is basically the same as the Vanicre...
2,1,I asked on the Vanicream Facebook site what is...
3,1,When Aveeno and then Alaffia changed their bod...
4,5,I’m a big believer in the Vanicream brand for ...
...,...,...
97,5,Worked great for my one year old with excema. ...
98,5,Thank you for such awesome product!
99,5,The baby formula must be an Amazon marketing s...
100,5,"I trust Vanicream with my own sensitive skin, ..."


In [169]:
# In order to do classification, it should be add a columns to denote which review is corresponding to which brand.

df_Aquaphor['Brand'] = 'Aquaphor'
df_Aveeno['Brand'] = 'Aveeno'
df_CeraVe['Brand'] = 'CeraVe'
df_Cetaphil['Brand'] = 'Cetaphil'
df_Era['Brand']= 'Era'
df_Eucerin['Brand']= 'Eucerin'
df_Mustela['Brand'] = 'Mustela'
df_Vanicream['Brand'] = 'Vanicream'

In [170]:
df_Aquaphor

,review rating,review text,Brand
0,1,This might be a fake aquaphor so people beware...,Aquaphor
1,4,Edit: we have continued to use Aquaphor from A...,Aquaphor
2,1,I received a very strange container. It had no...,Aquaphor
3,3,"Like others said, this aquaphor bought on amaz...",Aquaphor
4,4,So close to 5 stars. This product is AMAZING! ...,Aquaphor
...,...,...,...
4329,1,Full of chemicals! Do not use! Wish I could gi...,Aquaphor
4330,3,too oily,Aquaphor
4331,1,Very BAD ingredients... especially MINERAL OIL...,Aquaphor
4332,5,Works great for my baby. No fragrance and keep...,Aquaphor


# Data Cleaning 

Goal: Get the data in a clean, standard format for further analysis

## Apply the first round of text cleaning techniques
- make text lowercase 
- remove text in squre bracket,
- remove punctuation
- remove words containing numbers

In [171]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [172]:
# Because some of the review are number in  df_Aquaphor['review text'], so convert them into string type firstly. 

df_Aquaphor['review text'] = [str(reveiw) for reveiw in df_Aquaphor['review text'] ]
df_Aquaphor['review text'] = [reveiw.lower() for reveiw in df_Aquaphor['review text']]

In [173]:
# Applying the function for all datasets 

df_Aquaphor['review text'] = df_Aquaphor['review text'].apply(round1)
df_Aveeno['review text'] = df_Aveeno['review text'].apply(round1)
df_CeraVe['review text'] = df_CeraVe['review text'].apply(round1)
df_Cetaphil['review text'] = df_Cetaphil['review text'].apply(round1)
df_Era['review text']= df_Era['review text'].apply(round1)
df_Eucerin['review text']= df_Eucerin['review text'].apply(round1)
df_Mustela['review text'] = df_Mustela['review text'].apply(round1)
df_Vanicream['review text'] = df_Vanicream['review text'].apply(round1)

In [174]:
# Let's take a look at the updated text

df_Aquaphor['review text']

0       this might be a fake aquaphor so people beware...
1       edit we have continued to use aquaphor from am...
2       i received a very strange container it had no ...
3       like others said this aquaphor bought on amazo...
4       so close to  stars this product is amazing the...
                              ...                        
4329    full of chemicals do not use wish i could give...
4330                                             too oily
4331    very bad ingredients especially mineral oil mi...
4332    works great for my baby no fragrance and keep ...
4333    muy grande ya tiene un año y todavia le queda ...
Name: review text, Length: 4334, dtype: object

## Apply a second round of text cleaning techniques

- get rid of some additional punctuation
- remove non-sensical text that was missed the first time around

In [175]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [176]:
# Applying the function for all datasets 

df_Aquaphor['review text'] = df_Aquaphor['review text'].apply(round2)
df_Aveeno['review text'] = df_Aveeno['review text'].apply(round2)
df_CeraVe['review text'] = df_CeraVe['review text'].apply(round2)
df_Cetaphil['review text'] = df_Cetaphil['review text'].apply(round2)
df_Era['review text']= df_Era['review text'].apply(round2)
df_Eucerin['review text']= df_Eucerin['review text'].apply(round2)
df_Mustela['review text'] = df_Mustela['review text'].apply(round2)
df_Vanicream['review text'] = df_Vanicream['review text'].apply(round2)

In [177]:
# Let's take a look at the updated text

df_Aquaphor

,review rating,review text,Brand
0,1,this might be a fake aquaphor so people beware...,Aquaphor
1,4,edit we have continued to use aquaphor from am...,Aquaphor
2,1,i received a very strange container it had no ...,Aquaphor
3,3,like others said this aquaphor bought on amazo...,Aquaphor
4,4,so close to stars this product is amazing the...,Aquaphor
...,...,...,...
4329,1,full of chemicals do not use wish i could give...,Aquaphor
4330,3,too oily,Aquaphor
4331,1,very bad ingredients especially mineral oil mi...,Aquaphor
4332,5,works great for my baby no fragrance and keep ...,Aquaphor


### Check if there is missing value in the dataset 

In [178]:
# It shows that there is no missing value appearing in any dataset from below result. 
i = 0
for datset in [df_Aquaphor,df_Aveeno,df_CeraVe,df_Cetaphil,df_Era,df_Eucerin,df_Mustela,df_Vanicream]:
    df_isna= pd.DataFrame(dataset.isna().sum())
    print('{}th dataset:{}'.format(i,df_isna))
    i += 1
          

0th dataset:             0
review name  0
review text  0
1th dataset:             0
review name  0
review text  0
2th dataset:             0
review name  0
review text  0
3th dataset:             0
review name  0
review text  0
4th dataset:             0
review name  0
review text  0
5th dataset:             0
review name  0
review text  0
6th dataset:             0
review name  0
review text  0
7th dataset:             0
review name  0
review text  0


## Apply for a third round: remove stop words and conduct lemmatization 

In [179]:
# Define a function to deal with stop words 
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def round3(x):
    x = x.split()
    new_word = []
    for word in x:
        if word not in stop_words:
            new_word.append(lemmatizer.lemmatize(word))
            
    text_string = ' '.join(new_word)
    return text_string


In [180]:
df_Aquaphor['review text'] = df_Aquaphor['review text'].apply(round3)
df_Aveeno['review text'] = df_Aveeno['review text'].apply(round3)
df_CeraVe['review text'] = df_CeraVe['review text'].apply(round3)
df_Cetaphil['review text'] = df_Cetaphil['review text'].apply(round3)
df_Era['review text']= df_Era['review text'].apply(round3)
df_Eucerin['review text']= df_Eucerin['review text'].apply(round3)
df_Mustela['review text'] = df_Mustela['review text'].apply(round3)
df_Vanicream['review text'] = df_Vanicream['review text'].apply(round3)

In [181]:
# Let's take a look at the updated text

df_Aquaphor['review text'] 

0       might fake aquaphor people beware always purch...
1       edit continued use aquaphor amazon aside one w...
2       received strange container labeling white blue...
3       like others said aquaphor bought amazon differ...
4       close star product amazing reason giving star ...
                              ...                        
4329        full chemical use wish could give zero rating
4330                                                 oily
4331    bad ingredient especially mineral oil mineral ...
4332    work great baby fragrance keep skin moist woul...
4333    muy grande ya tiene un año todavia le queda po...
Name: review text, Length: 4334, dtype: object

# Save the cleaning dataset to CSV format for further exploration 

In [127]:
df_Aquaphor.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_Aquaphor.csv', index = False)
df_Aveeno.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_Aveeno.csv', index = False)
df_CeraVe.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_CeraVe.csv', index = False)
df_Cetaphil.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_Cetaphil.csv', index = False)
df_Era.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_Era.csv', index = False)
df_Eucerin.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_Eucerin.csv', index = False)
df_Mustela.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_Mustela.csv', index = False)
df_Vanicream.to_csv(r'D:\Springboard\Github\Data_NLP_Baby_Lotion\Processed Dataset\df_Vanicream.csv', index = False)